In [1]:
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np
import pandas as pd

In [2]:
#꼬꼬마분석기
kkma = Kkma() 

#text를 입력받아 Kkma.sentences()를 이용해 문장단위로 나눈 뒤 sentences로 리턴
def text2sentences(text):  
    sentences = kkma.sentences(text)
    for idx in range(0, len(sentences)):
        if len(sentences[idx]) <= 10:
            sentences[idx-1] += (' ' + sentences[idx])
            sentences[idx] = ''


    #text일 때 문장별로 리스트 만듦
    return sentences

In [3]:
#예시 줄거리

desc='''존재감 없이 살아온 20살 지원. 그는 자신의 생일날 삶을 마감하려 한다. 
하지만 우연히 만든 부캐로 인해 인생이 뒤바뀌고, 현실 ‘아싸’ 임지원은 부캐 ‘인싸’ 김시후로 살아가게 된다. 
점점 부캐의 삶에 빠져들수록 지원은 자신의 존재감을 잃어가고, 결국 부캐를 죽이기로 결심하는데... 
그는 부캐로 얻은 모든 것을 뒤로하고 다시 본인 자신으로 돌아갈 수 있을까.'''

In [4]:
sentences =text2sentences(desc)
print(sentences)

['존재 감 없이 살아온 20살 지원. 그는 자신의 생일날 삶을 마감하려 한다.', '하지만 우연히 만든 부캐로 인해 인생이 뒤바뀌고, 현실 ‘ 아 싸’ 임지원은 부캐 ‘ 인 싸’ 김 시 후로 살아가게 된다.', '점점 부캐의 삶에 빠져들 수록 지원은 자신의 존재 감을 잃어 가고, 결국 부캐를 죽이기로 결심하는데... 그는 부캐로 얻은 모든 것을 뒤로하고 다시 본인 자신으로 돌아갈 수 있을까.']


In [5]:
twitter = Twitter()

#불용어제거 수정하기
with open('C:/Users/user/CodingWorkspace/stopwords.txt', 'r') as f:
    list_file = f.readlines()
stopwords = list_file[0].split(",")


def get_nouns(sentences):
    nouns = []
    for sentence in sentences:
        if sentence is not '':
            nouns.append(' '.join([noun for noun in twitter.nouns(str(sentence))
                                  if noun not in stopwords and len(noun) >1])) # 한글자 제거
    return nouns

<>:12: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:12: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
C:\Users\user\AppData\Local\Temp/ipykernel_79308/3759735710.py:12: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if sentence is not '':


In [6]:
nouns = get_nouns(sentences)

In [20]:

nouns = get_nouns(sentences)
print(sentences[0])
print(nouns[0])
twitter.pos(nouns[2])


존재 감 없이 살아온 20살 지원. 그는 자신의 생일날 삶을 마감하려 한다.
존재 지원 생일 마감


[('점점', 'Noun'),
 ('부캐', 'Noun'),
 ('수록', 'Noun'),
 ('지원', 'Noun'),
 ('존재', 'Noun'),
 ('부캐', 'Noun'),
 ('결심', 'Noun'),
 ('부캐', 'Noun'),
 ('뒤', 'Noun'),
 ('로', 'Josa'),
 ('본인', 'Noun')]

In [8]:
tfidf = TfidfVectorizer()
cnt_vec = CountVectorizer()
graph_sentence = []

In [9]:
def build_words_graph(sentence):
    cnt_vec_mat = normalize(cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
    vocab = cnt_vec.vocabulary_
    return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}
    
words_graph, idx2word = build_words_graph(nouns)
print(idx2word)

{10: '존재', 11: '지원', 5: '생일', 2: '마감', 4: '부캐', 7: '인생', 12: '현실', 8: '임지원', 9: '점점', 6: '수록', 0: '결심', 1: '뒤로', 3: '본인'}


In [10]:
def get_ranks(graph, d=0.85): 
    A = graph
    matrix_size = A.shape[0]
    for id in range(matrix_size):
        A[id, id] = 0 
        link_sum = np.sum(A[:,id]) 
        if link_sum != 0:
            A[:, id] /= link_sum
        A[:, id] *= -d
        A[id, id] = 1

    B = (1-d) * np.ones((matrix_size, 1))
    ranks = np.linalg.solve(A, B) 
    return {idx: r[0] for idx, r in enumerate(ranks)}

In [11]:
word_rank_idx = get_ranks(words_graph)
print(word_rank_idx)

{0: 1.145543085154743, 1: 1.145543085154743, 2: 0.5882267698975936, 3: 1.1455430851547428, 4: 1.3945015757484505, 5: 0.5882267698975936, 6: 1.1455430851547428, 7: 0.7290611339693928, 8: 0.7290611339693926, 9: 1.145543085154743, 10: 1.2570730283872327, 11: 1.2570730283872327, 12: 0.7290611339693926}


In [12]:
sorted_word_rank_idx = sorted(word_rank_idx, key=lambda k: word_rank_idx[k], reverse=True)

In [13]:
# 키워드 수 조정
def keywords(word_num=10):

    keywords = []
    index=[]
    for idx in sorted_word_rank_idx[:word_num]:
        index.append(idx)

    #index.sort()
    for idx in index:
        keywords.append(idx2word[idx])
    print(keywords)

In [14]:
keywords()

['부캐', '존재', '지원', '결심', '뒤로', '점점', '본인', '수록', '인생', '임지원']
